In [1]:
# Importing all the necessary modules
import tensorflow as tf
from tensorflow import keras
from google.cloud import storage


from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import UpSampling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Input
from tensorflow.keras.optimizers import SGD, Adam, RMSprop
from tensorflow.keras.layers import Conv2D, Conv2DTranspose
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LeakyReLU, PReLU
from tensorflow.keras.layers import add



In [2]:
from io import BytesIO
import numpy as np
from tensorflow.python.lib.io import file_io

f = BytesIO(file_io.read_file_to_string("gs://ucsb_data/lowres.npy", binary_mode=True))
arr = np.load(f)
type(arr)

numpy.ndarray

In [3]:
# Residual block
def res_block_gen(model, kernal_size, filters, strides):
    
    gen = model
    
    model = Conv2D(filters = filters, kernel_size = kernal_size, strides = strides, padding = "same")(model)
    model = BatchNormalization(momentum = 0.5)(model)
    # Using Parametric ReLU
    model = PReLU(alpha_initializer='zeros', alpha_regularizer=None, alpha_constraint=None, shared_axes=[1,2])(model)
    model = Conv2D(filters = filters, kernel_size = kernal_size, strides = strides, padding = "same")(model)
    model = BatchNormalization(momentum = 0.5)(model)
        
    model = add([gen, model])
    
    return model

In [4]:
def up_sampling_block(model, kernal_size, filters, strides):
    '''
    In place of Conv2D and UpSampling2D we can also use Conv2DTranspose (Both are used for Deconvolution)
    Even we can have our own function for deconvolution
    model = Conv2DTranspose(filters = filters, kernel_size = kernal_size, strides = strides, padding = "same")(model)
    '''
    
    model = Conv2D(filters = filters, kernel_size = kernal_size, strides = strides, padding = "same")(model)
    model = UpSampling2D(size = 2)(model)
    model = LeakyReLU(alpha = 0.2)(model)
    
    return model

In [5]:
def discriminator_block(model, filters, kernel_size, strides):
    
    model = Conv2D(filters = filters, kernel_size = kernel_size, strides = strides, padding = "same")(model)
    model = BatchNormalization(momentum = 0.5)(model)
    model = LeakyReLU(alpha = 0.2)(model)
    
    return model

In [6]:

class Generator(object):

    def __init__(self, noise_shape):
        
        self.noise_shape = noise_shape

    def generator(self):
        
        gen_input = Input(shape = self.noise_shape)

        model = Conv2D(filters = 16, kernel_size = 9, strides = 1, padding = "same")(gen_input)
        model = PReLU(alpha_initializer='zeros', alpha_regularizer=None, alpha_constraint=None, shared_axes=[1,2])(model)

        gen_model = model

        # Using 16 Residual Blocks
        for index in range(16):
            model = res_block_gen(model, 3, 16, 1)

        model = Conv2D(filters = 16, kernel_size = 3, strides = 1, padding = "same")(model)
        model = BatchNormalization(momentum = 0.5)(model)
        model = add([gen_model, model])

        # Using 2 UpSampling Blocks
        for index in range(2):
            model = up_sampling_block(model, 3, 64, 1)

        model = Conv2D(filters = 1, kernel_size = 9, strides = 1, padding = "same")(model)
        model = Activation('tanh')(model)

        generator_model = Model(inputs = gen_input, outputs = model)

        return generator_model
    
image_shape = (64,64,1)

In [7]:
# Network Architecture is same as given in Paper https://arxiv.org/pdf/1609.04802.pdf
class Discriminator(object):

    def __init__(self, image_shape):
        
        self.image_shape = image_shape
    
    def discriminator(self):
        
        dis_input = Input(shape = self.image_shape)
        
        model = Conv2D(filters = 16, kernel_size = 3, strides = 1, padding = "same")(dis_input)
        model = LeakyReLU(alpha = 0.2)(model)
        
        model = discriminator_block(model, 8, 3, 2)
        model = discriminator_block(model, 16, 3, 1)
        model = discriminator_block(model, 16, 3, 2)
        model = discriminator_block(model, 32, 3, 1)
        model = discriminator_block(model, 32, 3, 2)
        model = discriminator_block(model, 64, 3, 1)
        model = discriminator_block(model, 64, 3, 2)
        
        model = Flatten()(model)
        model = Dense(64)(model)
        model = LeakyReLU(alpha = 0.2)(model)
       
        model = Dense(1)(model)
        model = Activation('sigmoid')(model) 
        
        discriminator_model = Model(inputs = dis_input, outputs = model)
        return discriminator_model

In [8]:
def get_gan_network(discriminator, shape, generator, optimizer):
    discriminator.trainable = False
    gan_input = Input(shape=shape)
    x = generator(gan_input)
    gan_output = discriminator(x)
    gan = Model(inputs=gan_input, outputs=[x,gan_output])
    gan.compile(loss=["binary_crossentropy", "binary_crossentropy"],
                loss_weights=[1., 1e-3],
                optimizer=optimizer)

    return gan

In [9]:
import numpy as np


#loading the lowres data ⬇️
from io import BytesIO
import numpy as np
from tensorflow.python.lib.io import file_io

f = BytesIO(file_io.read_file_to_string("gs://ucsb_data/lowres.npy", binary_mode=True))
X=np.load(f)

# Finding out where are nan values. Returns boolean array of same size

whnan=np.isnan(X)
whnan
X[whnan]=-1
X_train,X_test=X[:10000,:,:],X[-4300:,:,:]

#loading the highres data  ⬇️
f = BytesIO(file_io.read_file_to_string("gs://ucsb_data/highres.npy", binary_mode=True))
Y=np.load(f)
whnan=np.isnan(Y)
whnan
Y[whnan]=-1
Y_train,Y_test=Y[:10000,:,:],Y[-4300:,:,:]

In [10]:
X_train.shape

(10000, 16, 16)

In [11]:
Y_train.shape

(10000, 64, 64)

In [12]:
X_train = X_train.astype('float32') / 400.
X_test = X_test.astype('float32') / 400.
Y_train = Y_train.astype('float32') / 400.
Y_test = Y_test.astype('float32') / 400.
# add 1 extra dimension so single input vector looks like [[[]]]
x_train_lr_new = np.reshape(X_train, (len(X_train), 16, 16, 1))
x_test_lr_new = np.reshape(X_test, (len(X_test), 16, 16, 1))
x_train_hr_new = np.reshape(Y_train, (len(Y_train), 64, 64, 1))
x_test_hr_new = np.reshape(Y_test, (len(Y_test), 64, 64, 1))


In [13]:
x_train_lr_new.shape

(10000, 16, 16, 1)

In [14]:
x_train_hr_new.shape

(10000, 64, 64, 1)

In [15]:
x_test_hr_new.shape

(4300, 64, 64, 1)

In [16]:
x_test_lr_new.shape

(4300, 16, 16, 1)

In [17]:
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import mean_squared_error as mse

In [18]:
epochs=100 # epochs = 20000
batch_size=64

downscale_factor = 4
    
batch_count = int(x_train_hr_new.shape[0] / batch_size)
shape = (image_shape[0]//downscale_factor, image_shape[1]//downscale_factor, image_shape[2])
generator = Generator(shape).generator()
discriminator = Discriminator(image_shape).discriminator()

adam = Adam(lr=1E-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
generator.compile(loss="binary_crossentropy", optimizer=adam)
discriminator.compile(loss="binary_crossentropy", optimizer=adam)

shape = (image_shape[0]//downscale_factor, image_shape[1]//downscale_factor, 1)
gan = get_gan_network(discriminator, shape, generator, adam)

2021-09-29 14:57:28.232622: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN)to use the following CPU instructions in performance-critical operations:  AVX512F
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-09-29 14:57:28.251683: I tensorflow/core/platform/profile_utils/cpu_utils.cc:104] CPU Frequency: 2000095000 Hz
2021-09-29 14:57:28.252790: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x556f674cf6c0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2021-09-29 14:57:28.252824: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2021-09-29 14:57:28.255049: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [ ]:
hr_loss, lr_loss, gan_loss = [], [], []
m1, m2, m3 = [], [], []

for e in range(1, epochs+1):
    print ('-'*15, 'Epoch %d' % e, '-'*15)
    for i in range(batch_count):
        rand_nums = np.random.randint(0, x_train_hr_new.shape[0], size=batch_size)

        image_batch_hr =  np.stack(x_train_hr_new[rand_nums], axis=0)
        image_batch_lr = np.stack(x_train_lr_new[rand_nums], axis=0)
            
        generated_images_sr = generator.predict(image_batch_lr)

        real_data_Y = np.ones(batch_size) - np.random.random_sample(batch_size)*0.2
        fake_data_Y = np.random.random_sample(batch_size)*0.2

        discriminator.trainable = True

        d_loss_real = discriminator.train_on_batch(image_batch_hr, real_data_Y)
        d_loss_fake = discriminator.train_on_batch(generated_images_sr, fake_data_Y)
        #d_loss = 0.5 * np.add(d_loss_fake, d_loss_real)

        rand_nums = np.random.randint(0, x_train_hr_new.shape[0], size=batch_size)
    
        image_batch_hr =  np.stack(x_train_hr_new[rand_nums], axis=0)
        image_batch_lr = np.stack(x_train_lr_new[rand_nums], axis=0)

        gan_Y = np.ones(batch_size) - np.random.random_sample(batch_size)*0.2

        discriminator.trainable = False
        loss_gan = gan.train_on_batch(image_batch_lr, [image_batch_hr,gan_Y])

    print("Loss HR , Loss LR, Loss GAN")
    print(d_loss_real, d_loss_fake, loss_gan)
    
    hr_loss.append(d_loss_real)
    lr_loss.append(d_loss_fake)
    gan_loss.append(loss_gan)


--------------- Epoch 1 ---------------
Loss HR , Loss LR, Loss GAN
0.3457344174385071 0.3604010343551636 [0.07615480571985245, 0.07437014579772949, 1.7846577167510986]
--------------- Epoch 2 ---------------


In [ ]:
import matplotlib.pyplot as plt 

# rand_nums = np.random.randint(0, x_train_hr_new.shape[0], size=2)
rand_nums=[23]
image_batch_hr =  np.stack(x_train_hr_new[rand_nums], axis=0)
image_batch_lr = np.stack(x_train_lr_new[rand_nums], axis=0)
            
generated_images_sr = generator.predict(image_batch_lr)

In [ ]:
    fig, ax = plt.subplots(nrows=1, ncols=3,figsize=(15,15))
    

    ax[0].imshow(image_batch_lr[0])
    ax[0].title.set_text('Low Res')
    
    ax[1].imshow(generated_images_sr[0])
    ax[1].title.set_text('Generated Super Resution')
    
    ax[2].imshow(image_batch_hr[0])
    ax[2].title.set_text('Ground Truth (High Res)')


In [ ]:
plt.plot(d_loss_fake)

In [ ]:
len(gan_loss)

for a in gan_loss:
    print(a)

In [ ]:
plt.plot(gan_loss)
print(np.mean(gan_loss))